Elastic Search Script


In [2]:
!pip install torch torchvision --index-url https://download.pytorch.org/whl/cpu
!pip install pillow numpy elasticsearch tqdm
!pip install git+https://github.com/openai/CLIP.git
!pip install elasticsearch
!pip install requests beautifulsoup4 tqdm pillow
!pip install requests lxml tqdm
!pip install sentence-transformers
!pip install scikit-learn



Looking in indexes: https://download.pytorch.org/whl/cpu
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 960.5/960.5 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.3/65.3 kB 4.3 MB/s eta 0:00:00
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-niqer0yx
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-niqer0yx
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 1.8 MB/s eta 0:00:00
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369490 sha256=669566a7c2de385e9524e6e185df5bf3e8ae3a398a46d9d147a4fe351013df70
  Stored in directory: /tmp/pip-ephem-wheel-cache-pj9fczdx/wheels/35/3e/df/3d24cbfb3b6a06f17a2bfd7d1138900d4365d9028aa8f6e92f
Successfully built clip


In [3]:
import clip
import torch
from PIL import Image
import numpy as np
from elasticsearch import Elasticsearch, helpers
from sentence_transformers import SentenceTransformer
import numpy as np


client = Elasticsearch(
    "https://33cb0c87675645148676794cbec9680d.us-east-1.aws.found.io:443",
    api_key="a3FSbU5Kb0JJWUFJRE8wZWRYRmg6VDRSYjhROXVPU1JBSVhoemxsNHpZQQ=="
)

index_name = "multimodal_media_v2"
print(client.info().body)

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
MODEL, PREPROCESS = clip.load("ViT-L/14", device=DEVICE)  # 768-dim

print(client.indices.get_mapping(index=index_name))


# Load E5 model for long text
text_model = SentenceTransformer("intfloat/e5-base-v2")

def embed_text(text: str) -> np.ndarray:
    """
    Generate an embedding for long webpage text using E5 model.
    """
    # Clean + prefix (E5 expects a prompt style)
    text = text.strip()
    text = "passage: " + text
    emb = text_model.encode(text, normalize_embeddings=True)
    return np.array(emb)

# def embed_text(text: str) -> np.ndarray:
#     with torch.no_grad():
#         tokens = clip.tokenize([text]).to(DEVICE)
#         feats = MODEL.encode_text(tokens)
#         feats = feats / feats.norm(dim=-1, keepdim=True)
#     return feats[0].cpu().numpy().astype(np.float32)

def embed_image(path_or_pil) -> np.ndarray:
    img = path_or_pil if isinstance(path_or_pil, Image.Image) else Image.open(path_or_pil).convert("RGB")
    img = PREPROCESS(img).unsqueeze(0).to(DEVICE)
    with torch.no_grad():
        feats = MODEL.encode_image(img)
        feats = feats / feats.norm(dim=-1, keepdim=True)
    return feats[0].cpu().numpy().astype(np.float32)

def embed_text_clip(text: str) -> np.ndarray:
    """
    Generate an embedding using CLIP's text encoder.
    This ensures text and image vectors are in the same semantic space.
    """
    with torch.no_grad():
        tokens = clip.tokenize([text]).to(DEVICE)
        feats = MODEL.encode_text(tokens)
        feats = feats / feats.norm(dim=-1, keepdim=True)
    return feats[0].cpu().numpy().astype(np.float32)


def index_image(doc_id: str, title: str, url: str, vector):
    body = {
        "id": doc_id,
        "media_type": "image",
        "title": title,
        "url": url,
        "vector_clip": vector.tolist(),
        "metadata": {}
    }
    client.index(index=index_name, id=doc_id, document=body)
    print(f"Indexed image with id = {doc_id}")


# TEMPORARY TEST
if __name__ == "__main__":
    print(len(embed_text("hello world")))  # should print 768


{'name': 'instance-0000000006', 'cluster_name': '33cb0c87675645148676794cbec9680d', 'cluster_uuid': 'Eq0DnWFDQxmO_d_YkDYgLQ', 'version': {'number': '9.0.0', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '112859b85d50de2a7e63f73c8fc70b99eea24291', 'build_date': '2025-04-08T15:13:46.049795831Z', 'build_snapshot': False, 'lucene_version': '10.1.0', 'minimum_wire_compatibility_version': '8.18.0', 'minimum_index_compatibility_version': '8.0.0'}, 'tagline': 'You Know, for Search'}


100%|████████████████████████████████████████| 890M/890M [00:08<00:00, 107MiB/s]


{'multimodal_media_v2': {'mappings': {'properties': {'id': {'type': 'text', 'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}}, 'media_type': {'type': 'keyword'}, 'metadata': {'properties': {'length': {'type': 'long'}}}, 'title': {'type': 'text'}, 'url': {'type': 'keyword'}, 'vector_clip': {'type': 'dense_vector', 'dims': 768, 'index': True, 'similarity': 'cosine', 'index_options': {'type': 'int8_hnsw', 'm': 16, 'ef_construction': 100}}, 'vector_e5': {'type': 'dense_vector', 'dims': 768, 'index': True, 'similarity': 'cosine', 'index_options': {'type': 'int8_hnsw', 'm': 16, 'ef_construction': 100}}}}}}


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

768


In [4]:
from PIL import Image
import requests
from io import BytesIO

# Example image URL
image_url = "https://member.acg.aaa.com/content/dam/acg/images/home/pr-home/hero-splash-3.jpg"  #  replace with your image URL

# Download and open image from URL
response = requests.get(image_url)
img = Image.open(BytesIO(response.content)).convert("RGB")

# Generate vector
vec = embed_image(img)
print("Image embedding length:", len(vec))

# Index into Elasticsearch
index_image(
    doc_id="img_001",
    title="Test Image",
    url=image_url,
    vector=vec
)


Image embedding length: 768
Indexed image with id = img_001


In [5]:
from elasticsearch import Elasticsearch
import numpy as np

def search_multimodal(query: str, top_k: int = 5):
    """
    Performs multimodal search: compares text query embedding
    with both E5 (text) and CLIP (image) vectors.
    Returns combined results sorted by cosine similarity.
    """
    # Generate query embeddings
    query_vec_e5 = embed_text(query).tolist()
    query_vec_clip = embed_text_clip(query).tolist()  # optional fallback for consistency

    # --- Search text-based vectors (E5) ---
    e5_query = {
        "size": top_k,
        "query": {
            "script_score": {
                "query": {"exists": {"field": "vector_e5"}},
                "script": {
                    "source": "cosineSimilarity(params.query_vector, 'vector_e5') + 1.0",
                    "params": {"query_vector": query_vec_e5}
                }
            }
        }
    }

    # --- Search image-based vectors (CLIP) ---
    clip_query = {
        "size": top_k,
        "query": {
            "script_score": {
                "query": {"exists": {"field": "vector_clip"}},
                "script": {
                    "source": "cosineSimilarity(params.query_vector, 'vector_clip') + 1.0",
                    "params": {"query_vector": query_vec_clip}
                }
            }
        }
    }

    # Execute both searches
    e5_res = client.search(index=index_name, body=e5_query)
    clip_res = client.search(index=index_name, body=clip_query)

    # Combine both sets of hits
    combined_hits = []
    for hit in e5_res["hits"]["hits"]:
        combined_hits.append({
            "score": hit["_score"],
            "type": hit["_source"].get("media_type"),
            "title": hit["_source"].get("title"),
            "url": hit["_source"].get("url")
        })
    for hit in clip_res["hits"]["hits"]:
        combined_hits.append({
            "score": hit["_score"],
            "type": hit["_source"].get("media_type"),
            "title": hit["_source"].get("title"),
            "url": hit["_source"].get("url")
        })

    # Sort by score descending
    combined_hits = sorted(combined_hits, key=lambda x: x["score"], reverse=True)

    print(f"\n Search Results for: '{query}'")
    for item in combined_hits[:top_k]:
        print(f"[{item['type'].upper()}] {item['title']} → {item['url']} (score={item['score']:.3f})")

    return combined_hits


In [8]:

if __name__ == "__main__":
    # Example: text search instead of manual upload
    query = input("Enter your search text: ")
    results = search_multimodal(query, top_k=100)
    print(results)
    print("\n Search Results:")
    for i, r in enumerate(results, 1):
        print(f"{i}. [{r['score']:.4f}] {r['title']} → {r['url']}")


Enter your search text: broken car window

 Search Results for: 'broken car window'
[IMAGE] 175416163 → https://aaa.scene7.com/is/image/aaaautoclubsouthstage/meemic-auto-glass-repair?ts=1724055928251&bfc=on&network=on&dirt_feature=smart_imaging&dpr=off (score=0.6)
[IMAGE] 175416163 → https://aaa.scene7.com/is/image/aaaautoclubsouthstage/meemic-auto-glass-repair?ts=1724055928251&bfc=on&network=on&dirt_feature=smart_imaging&dpr=off (score=0.6)
[IMAGE] 104300948 → https://dev1.meemic.com/content/meemic/us/en/member-services/safety-information-center/auto-theft-prevention/_jcr_content/root/container/container/article-banner/image.coreimg.jpeg/1724055909559/auto-theft-article-hero.jpeg (score=0.5327)
[IMAGE] 5 Myths About Auto Theft → https://aaa.scene7.com/is/image/aaaautoclubsouthstage/auto-theft-1?ts=1732136180530&bfc=on&network=on&dirt_feature=smart_imaging&dpr=off (score=0.5141)
[IMAGE]  → https://dev1.meemic.com/content/meemic/us/en/why-meemic/blog/claims-safety/windshield-repair-or-r

In [ ]:
from xml.etree import ElementTree
from urllib.parse import urljoin
from bs4 import BeautifulSoup
from tqdm import tqdm
from PIL import Image
import requests
from io import BytesIO

def crawl_from_sitemap(sitemap_url: str, limit: int = 500):
    """
    Reads the sitemap XML, extracts <loc> URLs,
    fetches each page, extracts text/images,
    and indexes them into Elasticsearch with correct embeddings.
    """
    print(f" Reading sitemap: {sitemap_url}")
    resp = requests.get(sitemap_url, timeout=10)
    xml_root = ElementTree.fromstring(resp.content)

    urls = [elem.text for elem in xml_root.iter("{http://www.sitemaps.org/schemas/sitemap/0.9}loc")]
    print(f" Found {len(urls)} URLs in sitemap")

    count = 0
    for url in tqdm(urls[:limit], desc="Indexing pages"):
        try:
            r = requests.get(url, timeout=8)
            if "text/html" not in r.headers.get("Content-Type", ""):
                continue

            soup = BeautifulSoup(r.text, "html.parser")
            page_title = soup.title.string if soup.title else url
            page_text = " ".join(
                t.get_text(strip=True) for t in soup.find_all(["p", "h1", "h2", "h3"])
            )

            # --- Index Page Text (E5 Embedding) ---
            if len(page_text) > 100:
                vec = embed_text(page_text)  # E5 model
                client.index(index=index_name, id=f"page_{count}", document={
                    "id": f"page_{count}",
                    "media_type": "page",
                    "title": page_title,
                    "url": url,
                    "vector_e5": vec.tolist(),   # ✅ Correct field name
                    "metadata": {"length": len(page_text)}
                })
                count += 1

            # --- Index Images (CLIP Embedding) ---
            for img_tag in soup.find_all("img", src=True):
                img_url = urljoin(url, img_tag["src"])
                try:
                    img_resp = requests.get(img_url, timeout=5)
                    img = Image.open(BytesIO(img_resp.content)).convert("RGB")
                    vec = embed_image(img)  # CLIP model
                    index_image(
                        doc_id=f"img_{count}",
                        title=img_tag.get("alt", f"Image {count}"),
                        url=img_url,
                        vector=vec
                    )
                    count += 1
                except Exception:
                    pass

        except Exception as e:
            print(f" Skipping {url}: {e}")

    print(f"\n Finished indexing {count} items from sitemap.")


In [ ]:
if __name__ == "__main__":
    sitemap_url = input("Enter the sitemap url ")
    crawl_from_sitemap(sitemap_url, limit=500)


Enter the sitemap url https://dev1.meemic.com/sitemap.xml
 Reading sitemap: https://dev1.meemic.com/sitemap.xml
 Found 208 URLs in sitemap


Indexing pages:   0%|          | 0/208 [00:00<?, ?it/s]

Indexed image with id = img_1
Indexed image with id = img_2
Indexed image with id = img_3
Indexed image with id = img_4
Indexed image with id = img_5
Indexed image with id = img_6
Indexed image with id = img_7
Indexed image with id = img_8
Indexed image with id = img_9
Indexed image with id = img_10
Indexed image with id = img_11
Indexed image with id = img_12


Indexing pages:   0%|          | 1/208 [00:51<2:58:38, 51.78s/it]

Indexed image with id = img_13
Indexed image with id = img_15
Indexed image with id = img_16
Indexed image with id = img_17
Indexed image with id = img_18
Indexed image with id = img_19
Indexed image with id = img_20
Indexed image with id = img_21
Indexed image with id = img_22
Indexed image with id = img_23
Indexed image with id = img_24
Indexed image with id = img_25
Indexed image with id = img_26
Indexed image with id = img_27
Indexed image with id = img_28


...

Indexed image with id = img_1622
Indexed image with id = img_1624
Indexed image with id = img_1625
Indexed image with id = img_1626


Indexing pages: 100%|█████████▉| 207/208 [1:34:25<00:22, 22.61s/it]

Indexed image with id = img_1627
Indexed image with id = img_1629
Indexed image with id = img_1630
Indexed image with id = img_1631
Indexed image with id = img_1632
Indexed image with id = img_1633
Indexed image with id = img_1634
Indexed image with id = img_1635
Indexed image with id = img_1636
Indexed image with id = img_1637
Indexed image with id = img_1638
Indexed image with id = img_1639
Indexed image with id = img_1640
Indexed image with id = img_1641
Indexed image with id = img_1642
Indexed image with id = img_1643
Indexed image with id = img_1644


Indexing pages: 100%|██████████| 208/208 [1:35:35<00:00, 27.57s/it]

Indexed image with id = img_1645

 Finished indexing 1646 items from sitemap.
